# Predicciones con modelos LSTM

## 1) Cargamos el modelo desde el archivo .pt

In [ ]:
#@title Subir archivo .pt
try:
    from google.colab import files
    uploaded = files.upload()
except Exception:
    pass

In [ ]:
#@title Comprobar que el .pt contiene toda la información necesaria
import torch
path = "/content/lstm_rs_v3.pt"  
ckpt = torch.load(path, map_location="cpu", weights_only=False)
print("Keys en el checkpoint:", ckpt.keys())

# Verificaciones mínimas
req = {"model_state", "config", "stoi", "itos"}
missing = req - set(ckpt.keys())
print("Faltan:", missing if missing else "Nada")

# Tokens especiales y parámetros críticos
for tok in ["<unk>", "<bos>"]:
    assert tok in ckpt["stoi"], f"Falta el token {tok} en el vocabulario."
print("seq_len:", ckpt["config"].get("seq_len"))
print("tie_weights:", ckpt["config"].get("tie_weights"))


Keys en el checkpoint: dict_keys(['model_state', 'model_class', 'config', 'metrics_test', 'stoi', 'itos', 'vocab_size', 'created_at', 'env'])
Faltan: Nada
seq_len: 16
tie_weights: True


In [3]:
#@title Comprobación extra: vocab_size = len(stoi)
assert ckpt["vocab_size"] == len(ckpt["stoi"]), "vocab_size no coincide con len(stoi)"


In [6]:
#@title Recreamos la arquitectura ChordLSTM y cargamos el modelo
import torch, torch.nn as nn

class ChordLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size, num_layers, dropout, tie_weights=False):
        super().__init__()
        self.emb = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_size, num_layers=num_layers,
                           batch_first=True, dropout=dropout if num_layers>1 else 0.0)
        self.dropout = nn.Dropout(dropout)
        self.tie_weights = tie_weights
        if tie_weights:
            self.proj = nn.Linear(hidden_size, embedding_dim, bias=False) if hidden_size != embedding_dim else nn.Identity()
            self.decoder = nn.Linear(embedding_dim, vocab_size, bias=False)
            self.decoder.weight = self.emb.weight
        else:
            self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, x):
        e = self.emb(x)                 # (B, T, E)
        o, _ = self.rnn(e)              # (B, T, H)
        h = self.dropout(o[:, -1, :])   # (B, H)
        return self.decoder(self.proj(h)) if self.tie_weights else self.fc(h)  # (B, V)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def load_best_checkpoint(path):
    try:
        ckpt = torch.load(path, map_location="cpu")  # intenta modo seguro (weights_only=True)
    except Exception:
        ckpt = torch.load(path, map_location="cpu", weights_only=False)  # fallback
    cfg  = ckpt["config"]
    model = ChordLSTM(
        vocab_size=len(ckpt["stoi"]),
        embedding_dim=cfg["embedding_dim"],
        hidden_size=cfg["hidden_size"],
        num_layers=cfg["num_layers"],
        dropout=cfg["dropout"],
        tie_weights=cfg.get("tie_weights", False),
    )
    model.load_state_dict(ckpt["model_state"], strict=True)
    model.to(device).eval()
    return model, ckpt["stoi"], ckpt["itos"], cfg, ckpt.get("metrics_test", None)

model, stoi, itos, cfg, metrics = load_best_checkpoint("/content/lstm_rs_v3.pt") # "lstm_rs_v3.pt" / "lstm_rs_v2.pt"
print("Cargado. Métricas (test) guardadas:", metrics)


Cargado. Métricas (test) guardadas: {'loss': 2.252060778257626, 'ppl': 9.50730811622195, 'Top@1': 0.45036420395421434, 'Top@3': 0.6936524453818116, 'Top@5': 0.7761706556043317, 'MRR': 0.5956792587544246}


## 2) Definimos la función de predicción

In [7]:
#@title Definicion de `predict_next()` usando el chekpoint empaquetado
import torch.nn.functional as F

@torch.inference_mode()
def predict_next(model, stoi, itos, context_tokens, seq_len, k=5):
    unk_id = stoi.get("<unk>")
    if unk_id is None:
      raise KeyError("El vocabulario no contiene '<unk>'.")
    bos_id = stoi.get("<bos>")

    ids = [stoi.get(t, unk_id) for t in context_tokens]
    if len(ids) < seq_len and bos_id is not None:
        ids = [bos_id] * (seq_len - len(ids)) + ids
    else:
        ids = ids[-seq_len:]

    x = torch.tensor(ids, dtype=torch.long, device=device).unsqueeze(0)
    logits = model(x)                           # (1, V)
    probs  = F.softmax(logits[0], dim=-1)       # (V,)
    topk   = torch.topk(probs, k)
    return [(itos[i.item()], float(p.item())) for i,p in zip(topk.indices, topk.values)]


## 3) Evaluamos el rendimiento del modelo con algunas progresiones clásicas de acordes

#### Progresiones clásicas en tonalidad mayor

In [10]:
predict_next(model, stoi, itos, ["I"], seq_len=cfg["seq_len"], k=5)

[('I', 0.728754460811615),
 ('bii', 0.08350379765033722),
 ('io', 0.06358400732278824),
 ('vii', 0.02492613159120083),
 ('VII', 0.020677480846643448)]

In [11]:
#@title ["II", "V7"] > "I"
predict_next(model, stoi, itos, ["ii", "V7"], seq_len=cfg["seq_len"], k=5) # Top2

[('V7', 0.19316720962524414),
 ('I', 0.1889045089483261),
 ('bii', 0.12631601095199585),
 ('Vsub/VII', 0.0901450663805008),
 ('#IV7', 0.07938558608293533)]

In [12]:
#@title pop-punk ["I", "V"] >  "vi" "IV"
predict_next(model, stoi, itos, ["I", "V"], seq_len=cfg["seq_len"], k=10) # No acierta

[('V', 0.3473909795284271),
 ('I', 0.11748773604631424),
 ('bii', 0.10781167447566986),
 ('II', 0.07944024354219437),
 ('vo', 0.06364025920629501),
 ('Vsub/VII', 0.04106232896447182),
 ('VII', 0.024579769000411034),
 ('vii', 0.02164432778954506),
 ('V/VII', 0.020316386595368385),
 ('io', 0.01924402080476284)]

In [13]:
#@title pop-punk ["I", "V", "vi"] > "IV"
predict_next(model, stoi, itos, ["I", "V", "vi"], seq_len=cfg["seq_len"], k=10) # No acierta

[('II', 0.3430151641368866),
 ('V/V', 0.14899735152721405),
 ('vii', 0.08112698048353195),
 ('V', 0.06390789896249771),
 ('II7', 0.04675060883164406),
 ('bIII', 0.025602517649531364),
 ('bviio', 0.024007735773921013),
 ('vi', 0.020670782774686813),
 ('bii', 0.018382836133241653),
 ('Vsub/VII', 0.01828482747077942)]

In [15]:
#@title ["I", "IV"] > "I"
predict_next(model, stoi, itos, ["I", "IV"], seq_len=cfg["seq_len"], k=5) # top 4

[('IV', 0.3151707947254181),
 ('VII', 0.10519496351480484),
 ('iii', 0.05927819758653641),
 ('I', 0.057974252849817276),
 ('bIII', 0.05760975554585457)]

In [18]:
#@title ["IV", "iv"] > "I" (@Top1)
predict_next(model, stoi, itos, ["IV", "iv"], seq_len=cfg["seq_len"], k=6)

[('iii', 0.2716154456138611),
 ('iv', 0.21452674269676208),
 ('V7', 0.04864968731999397),
 ('bIII7', 0.039231523871421814),
 ('VII', 0.038391176611185074),
 ('I', 0.035610079765319824)]

In [19]:
#@title ["i", "ii", "iii"] > "ii" o "IV"
predict_next(model, stoi, itos, ["i", "ii", "iii"], seq_len=cfg["seq_len"], k=5) # top3 top5

[('iii', 0.7038135528564453),
 ('bii', 0.06771917641162872),
 ('ii', 0.040831223130226135),
 ('VII', 0.03330032899975777),
 ('IV', 0.029631592333316803)]

In [ ]:
#@title ["vi", "ii"] > "V7"
predict_next(model, stoi, itos, ["vi", "ii"], seq_len=cfg["seq_len"], k=5) #top4

[('ii', 0.43206366896629333),
 ('bii', 0.18932126462459564),
 ('bIII7', 0.16770334541797638),
 ('V7', 0.1022372841835022),
 ('iii', 0.022949496284127235)]

In [21]:
#@title ["I", "bVII"] > "I"
predict_next(model, stoi, itos, ["I", "bVII"], seq_len=cfg["seq_len"], k=5) #top1

[('I', 0.2037191390991211),
 ('bii', 0.14785631000995636),
 ('IV', 0.08624669909477234),
 ('V7', 0.08021286875009537),
 ('vii', 0.07252342998981476)]

#### Progresiones clásicas en tonalidad menor:

In [22]:
#@title ["iiø", "V7"] > "i"
predict_next(model, stoi, itos, ["iiø","V7"], seq_len=cfg["seq_len"], k=5) # No acierta

[('V7', 0.34982505440711975),
 ('bii', 0.12342175841331482),
 ('vo', 0.09434334188699722),
 ('#IV7', 0.0677952766418457),
 ('I', 0.0671551302075386)]

In [24]:
#@title ["VI", "iiø"] > "V7"
predict_next(model, stoi, itos, ["VI","iiø"], seq_len=cfg["seq_len"], k=5) # top1

[('V7', 0.4765273928642273),
 ('vo', 0.11953472346067429),
 ('bii', 0.0630049929022789),
 ('iiø', 0.024293364956974983),
 ('bII7', 0.022607818245887756)]

In [25]:
#@title ["i", "iv"] > "V7"
predict_next(model, stoi, itos, ["i","iv"], seq_len=cfg["seq_len"], k=5) # top2

[('iv', 0.4768180251121521),
 ('V7', 0.1670931577682495),
 ('bIII7', 0.05279575660824776),
 ('i', 0.042017169296741486),
 ('iii', 0.034302033483982086)]

#### Dominantes secundatios y sustitutos

In [35]:
#@title ["i", "V/IV"] > "iv"
predict_next(model, stoi, itos, ["i", "V/IV"], seq_len=cfg["seq_len"], k=5) # top1

[('iv', 0.5058850646018982),
 ('IV', 0.46033164858818054),
 ('ivø', 0.01178439985960722),
 ('IV7', 0.009501107037067413),
 ('vii', 0.002293897559866309)]

In [27]:
#@title ["I", "Vsub/V"] > "V7"
predict_next(model, stoi, itos, ["I", "Vsub/V"], seq_len=cfg["seq_len"], k=5) # top1

[('V7', 0.7820895314216614),
 ('V', 0.10662396252155304),
 ('bii', 0.03307785838842392),
 ('vo', 0.02601395547389984),
 ('io', 0.010889648459851742)]

In [30]:
#@title ["I", "biio"] > "ii"
predict_next(model, stoi, itos, ["I", "biio"], seq_len=cfg["seq_len"], k=5) # top1

[('ii', 0.44632476568222046),
 ('vo', 0.15586133301258087),
 ('bii', 0.09905041754245758),
 ('biio', 0.06002334505319595),
 ('Vsub/VII', 0.046702779829502106)]

#### Caso de uso: componiendo una progresión de acordes escogiendo entre las sugerencias de predict_next()
secuencia resultante: i → V/V → V7 → i

In [36]:
predict_next(model, stoi, itos, ["i"], seq_len=cfg["seq_len"], k=5)

[('i', 0.34708404541015625),
 ('bIII7', 0.2831963896751404),
 ('bii', 0.11497221142053604),
 ('biii', 0.04154810681939125),
 ('bIII', 0.023175673559308052)]

In [ ]:
predict_next(model, stoi, itos, ["i", "V/V"], seq_len=cfg["seq_len"], k=5)

[('V', 0.7204613089561462),
 ('V7', 0.17053626477718353),
 ('v', 0.09626950323581696),
 ('bii', 0.005243147257715464),
 ('bvio', 0.0018288219580426812)]

In [ ]:
predict_next(model, stoi, itos, ["i", "V/V", "V7"], seq_len=cfg["seq_len"], k=5)


[('V7', 0.3577346205711365),
 ('bii', 0.08322422951459885),
 ('i', 0.06308762729167938),
 ('#IV7', 0.06268736720085144),
 ('vo', 0.04323260858654976)]

## 4) Definimos StatefulPredictor: predictor con memoria

In [ ]:
# @title Stateful predictor para uso interactivo paso a paso
import torch, torch.nn.functional as F
from torch import nn

class StatefulPredictor:
    def __init__(self, model, stoi, itos, device, start_with_bos=True):
        self.model, self.stoi, self.itos = model.eval(), stoi, itos
        self.device = device
        self.start_with_bos = start_with_bos
        self.h = None  # (h,c) en LSTM; h en GRU

    def reset(self):
        """
        Reinicia el estado oculto.
        Si start_with_bos=True, inyecta el token <bos> para marcar el inicio.
        """
        self.h = None
        if self.start_with_bos and "<bos>" in self.stoi:
            self._step("<bos>")  # marcador de inicio

    @torch.inference_mode()
    def _step(self, token):
        """
        Propaga un token y actualiza el estado oculto.
        Devuelve los logits (no normalizados) para el SIGUIENTE token.
        """
        tid = torch.tensor([[self.stoi.get(token, self.stoi["<unk>"])]], device=self.device)
        e = self.model.emb(tid)                            # (1,1,E)
        out, self.h = self.model.rnn(e, self.h)            # (1,1,H)
        last = out[:, -1, :]                               # (1,H)
        last = self.model.dropout(last)                    # ruta idéntica a forward
        logits = (
            self.model.decoder(self.model.proj(last))
            if getattr(self.model, "tie_weights", False)
            else self.model.fc(last)
        )                                                  # (1,V)
        return logits[0]                                   # (V,)

    def add(self, token, k=5):
        """
        Añade un nuevo acorde al contexto y devuelve predicción para el siguiente.
        """
        logits = self._step(token)
        probs = F.softmax(logits, dim=-1)
        topk = torch.topk(probs, k)
        return [(self.itos[i.item()], float(p.item())) for i,p in zip(topk.indices, topk.values)]


    def suggest_window(self, context_tokens, seq_len, k=5):
      """
      Emula EXACTAMENTE la lógica de predict_next():
      - recorta al último seq_len
      - acolcha con <bos> a la izquierda hasta seq_len si hace falta
      """
      self.h = None
      # No inyectamos <bos> automático aquí:
      # ignoramos self.start_with_bos a propósito

      # recorte/acolchado
      ctx = list(context_tokens)
      if len(ctx) >= seq_len:
          ctx = ctx[-seq_len:]
          bos_to_add = 0
      else:
          bos_to_add = seq_len - len(ctx)

      preds = None
      # 1) Acolchar con <bos>
      if "<bos>" in self.stoi:
          for _ in range(bos_to_add):
              preds = self.add("<bos>", k=k)
      # 2) Alimentar el contexto real
      for t in ctx:
          preds = self.add(t, k=k)
      # La última llamada produce la sugerencia para el siguiente
      return preds

    def suggest(self, context_tokens, k=5):
        """
        Modo 'streaming':
        1. Resetea el estado
        2. Consume el nuevo contexto completo
        3. Devuelve la predicción para el siguiente acorde
        - No trunca seq_len ni acolcha con múltiples <bos>
        """
        self.reset()
        preds = None
        for t in context_tokens:
            preds = self.add(t, k=k)  # la última llamada predice el siguiente
        return preds

In [ ]:
sp = StatefulPredictor(model, stoi, itos, device)
sp.suggest(["ii","V"], k=5)

[('I', 0.43416544795036316),
 ('ii', 0.27106398344039917),
 ('iii', 0.07574915885925293),
 ('V', 0.04748965799808502),
 ('V7', 0.030107686296105385)]

### Prueba rápida de consistencia
Podemos comprobar como si utilizamos `sp.suggest_window()` forzamos que el tamaño de la ventana (seq_len) sea igual al utilizado por `predict_next()`.

Sin embargo sp.sugest() o sp.add() no utilizan un tamaño de contexto fijo.

In [ ]:
# A) Función predict_next()
predA = predict_next(model, stoi, itos, ["I"], seq_len=cfg["seq_len"], k=5)

# B) Stateful emulando ventana fija
sp = StatefulPredictor(model, stoi, itos, device, start_with_bos=False)
predB = sp.suggest_window(["I"], seq_len=cfg["seq_len"], k=5)

# C) Stateful sin ventana fija
predC = sp.suggest(["I"], k=5)

print("predict_next:", predA)
print("stateful(win):", predB)
print("stateful(free):", predC)


predict_next: [('I', 0.728754460811615), ('bii', 0.08350379765033722), ('io', 0.06358400732278824), ('vii', 0.02492613159120083), ('VII', 0.020677480846643448)]
stateful(win): [('I', 0.7287544012069702), ('bii', 0.08350386470556259), ('io', 0.06358397752046585), ('vii', 0.024926118552684784), ('VII', 0.020677490159869194)]
stateful(free): [('I', 0.47631093859672546), ('ii', 0.080355204641819), ('vi', 0.04353588446974754), ('iii', 0.02913953922688961), ('IV', 0.029027318581938744)]


#### Secuencia de acordes escogiendo entre los sugeridos: I → VII7 → iii → V7 → I
utilizando `sp.suggest_window()`

In [ ]:
sp.suggest(["I", "V/iii"], k=5)

[('VI7', 0.08308775722980499),
 ('Vsub/IV', 0.07835402339696884),
 ('V/II', 0.06114249676465988),
 ('V/VII', 0.05947471410036087),
 ('V7', 0.049347199499607086)]

In [ ]:
sp.suggest_window(["I", "V/iii", "iii"], seq_len=cfg["seq_len"], k=5)

[('iii', 0.5743696093559265),
 ('bii', 0.08598511666059494),
 ('VII', 0.062040168792009354),
 ('II', 0.05795701593160629),
 ('ii', 0.045799002051353455)]

In [ ]:
sp.suggest_window(["I", "VII7", "iii", 'V7'], seq_len=cfg["seq_len"], k=5)


[('V7', 0.28407374024391174),
 ('iii', 0.23797465860843658),
 ('I', 0.1087404116988182),
 ('bii', 0.05222799628973007),
 ('#IV7', 0.0371098667383194)]

#### Secuencia de acordes (2) escogiendo entre los sugeridos: I → I7 → IV→ iv
utilizando `sp.add()`

In [ ]:
sp.reset()

In [ ]:
sp.add('I', k=5)

[('I', 0.47631093859672546),
 ('ii', 0.080355204641819),
 ('vi', 0.04353588446974754),
 ('iii', 0.02913953922688961),
 ('IV', 0.029027318581938744)]

In [ ]:
sp.add('ii')

[('V7', 0.5520331263542175),
 ('ii', 0.1373572051525116),
 ('I', 0.05468360334634781),
 ('biiio', 0.0536913238465786),
 ('iii', 0.05137204751372337)]

In [ ]:
sp.add('iii')

[('ii', 0.267689049243927),
 ('IV', 0.22527460753917694),
 ('iii', 0.09536802023649216),
 ('V/II', 0.08170545101165771),
 ('vi', 0.045558806508779526)]

In [ ]:
sp.add('ii')

[('I', 0.6592168211936951),
 ('iii', 0.15768370032310486),
 ('V7', 0.0501166470348835),
 ('ii', 0.03659089654684067),
 ('v', 0.014157912693917751)]

## Optamos por un enfoque *stateless* (`predict_next`) para la API

**Idea central:** cada interacción del cliente envía el contexto completo y recibe las **Top-k** propuestas del siguiente acorde. El servidor **no mantiene estado** entre peticiones.

### Ventajas clave
- **Simplicidad de implementación:** sin gestionar `h/c` del RNN, `reset()`, ni sincronización de estado entre sesiones.
- **Reproducibilidad e idempotencia:** misma entrada ⇒ misma salida (sin depender de un estado oculto previo).
- **Coherencia con la evaluación:** `predict_next` replica la **ventana fija** (`seq_len`) y el acolchado con `<bos>`, por lo que las predicciones son comparables con validación/test.
- **Escalabilidad y arquitectura limpia:** servidor *sin estado* (stateless), fácil *horizontal scaling*, *load balancing* y *cacheo* de respuestas.
- **Coste/latencia asumibles:** con `seq_len=16` y un LSTM pequeño, recomputar la ventana en cada paso es barato, incluso en **CPU** (no requiere GPU/CUDA).
- **Robustez operativa:** reiniciar tras un error del usuario es trivial (no hay “estado corrupto” en memoria del servidor).
- **Seguridad/aislamiento:** menos superficie de fallo y sin compartir estados entre usuarios.

### Qué perdemos vs. *stateful*
- **Eficiencia por paso:** en *stateful* el coste por token es O(1); aquí es O(`seq_len`). En nuestro caso el impacto es despreciable.
- **Contexto > `seq_len`:** el modo *streaming* con estado puede aprovechar contextos más largos (aunque el modelo fue entrenado con ventana fija).

### Cuándo reconsiderar *stateful*
- Generación **muy larga** o **masiva** (beam grande / muestreos múltiples).
- Requisitos de **latencia ultra-baja** y `seq_len` elevado.
- Necesidad real de contexto que exceda `seq_len`.

**Decisión:** dado nuestro caso (LSTM pequeño, `seq_len=16`, interacción paso a paso), el enfoque **stateless** ofrece el mejor equilibrio entre **simplicidad**, **consistencia** y **rendimiento**.


## 4) Prototipo: explorando posibilidades para poner el modelo en producción

In [ ]:
# @title Bucle mínimo funcional (mejorado)
k = 10
context = []

print("Construye tu progresión. Escribe 'exit' para terminar.\n")

while True:
    if context:
        # Mostrar la secuencia actual y sugerencias para el siguiente
        preds = predict_next(model, stoi, itos, context, seq_len=cfg["seq_len"], k=k)
        print(f"Secuencia: {', '.join(context)}")
        print("Sugerencias:")
        for i, (ch, p) in enumerate(preds, 1):
            print(f"  {i}. {ch}  ({p:.3f})")
    else:
        # Hasta que el usuario escriba el primer acorde, no sugerimos nada
        print("Introduce el primer acorde para empezar.")

    choice = input("Elige acorde (o 'exit'): ").strip()
    if choice.lower() == "exit":
        break
    if choice not in stoi:
        print("Acorde fuera del vocabulario; prueba otra vez.\n")
        continue

    context.append(choice)
    print()  # línea en blanco para separar iteraciones


Construye tu progresión. Escribe 'exit' para terminar.

Introduce el primer acorde para empezar.
Elige acorde (o 'exit'): i

Secuencia: i
Sugerencias:
  1. i  (0.347)
  2. bIII7  (0.283)
  3. bii  (0.115)
  4. biii  (0.042)
  5. bIII  (0.023)
  6. bviio  (0.018)
  7. bVI  (0.017)
  8. V  (0.013)
  9. Vsub/VII  (0.013)
  10. bvi  (0.012)


## 5) Roadmap
- Incluir reranking para evitar repeticiones.
- Opcion para ajustarse o no a una tonalidad dada.
- Armar API Rest con Flask o FastAPI (probar en puerto local)
- Incluir transcripcion C, Dm → I, ii → C, Dm para que el usuario introduzca y reciba acordes en notacion americana.
